In [1]:
import json

businessFilePath = './yelp_academic_dataset_business.json'
reviewFilePath = './yelp_academic_dataset_review.json'

In [2]:
from peewee import *
from playhouse.sqlite_ext import SqliteExtDatabase

db = SqliteExtDatabase('yelp.db')

class BaseModel(Model):
    class Meta:
        database = db

class Business(BaseModel):
#[u'city', u'neighborhood', u'name', u'business_id', u'longitude', u'hours', 
# u'state', u'postal_code', u'categories', u'stars', u'address', u'latitude',
# u'review_count', u'attributes', u'type', u'is_open']
    business_id = FixedCharField(unique=True)
    stars = FloatField(index=True)
    review_count = IntegerField(index=True)
    name = CharField()
    state = FixedCharField(index=True)
    city = FixedCharField(index=True)
    
class Category(BaseModel):
    business = ForeignKeyField(Business, related_name='categories')
    name = FixedCharField(index=True)

class Review(BaseModel):
# [u'funny', u'user_id', u'review_id', u'text', u'business_id', u'stars',
# u'date', u'useful', u'type', u'cool']
    review_id = FixedCharField(unique=True)
    business = ForeignKeyField(Business, related_name='reviews')
    stars = FloatField(index=True)
    text = TextField()
    
db.connect()
db.create_tables([Business, Review, Category])

In [3]:
n = 0
_n = 0
bids = set([])
for business in open(businessFilePath):
    _n += 1
    if _n % 1000 == 0:
        print '%.2f%%' % (100.0 * _n / 144072), 
    business = json.loads(business)
    if business['categories'] == None or 'Restaurants' not in business['categories']:
        continue
    businessObj = Business(**{k:business[k] for k in ['business_id', 'stars', 'name', 'review_count', 'state', 'city']})
    businessObj.save()
    bids.add(business['business_id'])
    for category in business['categories']:
        if category == 'Restaurants':
           continue
        categoryObj = Category(name=category, business=businessObj)
        categoryObj.save()
    
    n += 1

print
print n
        

0.69% 1.39% 2.08% 2.78% 3.47% 4.16% 4.86% 5.55% 6.25% 6.94% 7.64% 8.33% 9.02% 9.72% 10.41% 11.11% 11.80% 12.49% 13.19% 13.88% 14.58% 15.27% 15.96% 16.66% 17.35% 18.05% 18.74% 19.43% 20.13% 20.82% 21.52% 22.21% 22.91% 23.60% 24.29% 24.99% 25.68% 26.38% 27.07% 27.76% 28.46% 29.15% 29.85% 30.54% 31.23% 31.93% 32.62% 33.32% 34.01% 34.70% 35.40% 36.09% 36.79% 37.48% 38.18% 38.87% 39.56% 40.26% 40.95% 41.65% 42.34% 43.03% 43.73% 44.42% 45.12% 45.81% 46.50% 47.20% 47.89% 48.59% 49.28% 49.98% 50.67% 51.36% 52.06% 52.75% 53.45% 54.14% 54.83% 55.53% 56.22% 56.92% 57.61% 58.30% 59.00% 59.69% 60.39% 61.08% 61.77% 62.47% 63.16% 63.86% 64.55% 65.25% 65.94% 66.63% 67.33% 68.02% 68.72% 69.41% 70.10% 70.80% 71.49% 72.19% 72.88% 73.57% 74.27% 74.96% 75.66% 76.35% 77.04% 77.74% 78.43% 79.13% 79.82% 80.52% 81.21% 81.90% 82.60% 83.29% 83.99% 84.68% 85.37% 86.07% 86.76% 87.46% 88.15% 88.84% 89.54% 90.23% 90.93% 91.62% 92.31% 93.01% 93.70% 94.40% 95.09% 95.79% 96.48% 97.17% 97.87% 98.56% 99.26% 99.95%
48485


In [4]:
n = 0
_n = 0
for review in open(reviewFilePath):
    _n += 1
    if _n % 10000 == 0:
        print '%.2f%%' % (_n / 41531.500), 
    review = json.loads(review)
    if not review['business_id'] in bids:
        continue
    businessObj = Business.get(business_id = review['business_id'])
    attrs = {k:review[k] for k in ['review_id', 'stars', 'text']}
    attrs['business'] = businessObj
    reviewObj = Review(**attrs)
    reviewObj.save()
    
    n += 1
print 'done'
print n

0.24% 0.48% 0.72% 0.96% 1.20% 1.44% 1.69% 1.93% 2.17% 2.41% 2.65% 2.89% 3.13% 3.37% 3.61% 3.85% 4.09% 4.33% 4.57% 4.82% 5.06% 5.30% 5.54% 5.78% 6.02% 6.26% 6.50% 6.74% 6.98% 7.22% 7.46% 7.70% 7.95% 8.19% 8.43% 8.67% 8.91% 9.15% 9.39% 9.63% 9.87% 10.11% 10.35% 10.59% 10.84% 11.08% 11.32% 11.56% 11.80% 12.04% 12.28% 12.52% 12.76% 13.00% 13.24% 13.48% 13.72% 13.97% 14.21% 14.45% 14.69% 14.93% 15.17% 15.41% 15.65% 15.89% 16.13% 16.37% 16.61% 16.85% 17.10% 17.34% 17.58% 17.82% 18.06% 18.30% 18.54% 18.78% 19.02% 19.26% 19.50% 19.74% 19.98% 20.23% 20.47% 20.71% 20.95% 21.19% 21.43% 21.67% 21.91% 22.15% 22.39% 22.63% 22.87% 23.11% 23.36% 23.60% 23.84% 24.08% 24.32% 24.56% 24.80% 25.04% 25.28% 25.52% 25.76% 26.00% 26.25% 26.49% 26.73% 26.97% 27.21% 27.45% 27.69% 27.93% 28.17% 28.41% 28.65% 28.89% 29.13% 29.38% 29.62% 29.86% 30.10% 30.34% 30.58% 30.82% 31.06% 31.30% 31.54% 31.78% 32.02% 32.26% 32.51% 32.75% 32.99% 33.23% 33.47% 33.71% 33.95% 34.19% 34.43% 34.67% 34.91% 35.15% 35.39% 35.64% 35.88

In [12]:
b = Business.get(business_id = '4P-vTvE6cncJyUyLh73pxw')

In [13]:
b.name

u'London Pickle Works'

In [16]:
from operator import *
[x.name for x in b.categories]

[u'Nightlife', u'American (Traditional)', u'Pubs', u'Bars']

In [17]:
b.reviews

<class '__main__.Review'> SELECT "t1"."id", "t1"."review_id", "t1"."business_id", "t1"."stars", "t1"."text" FROM "review" AS t1 WHERE ("t1"."business_id" = ?) [11785]

In [18]:
len(b.reviews)

23

In [22]:
b.reviews[0].text

u'This place is a area staple! Been around for years and not much has changed - I see this as a good thing! Stable and reliable!\n\nMy family goes every year for St. Pattys Day corn beef! Very nice place for a bar night dinner, or to catch up with some friends over drinks!'